In [1]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
import matplotlib.pyplot as plt

objc[46449]: Class CaptureDelegate is implemented in both /Users/alice/opt/anaconda3/envs/new-env/lib/python3.10/site-packages/cv2/cv2.abi3.so (0x141ea2538) and /Users/alice/opt/anaconda3/envs/new-env/lib/python3.10/site-packages/mediapipe/.dylibs/libopencv_videoio.3.4.16.dylib (0x1260d0860). One of the two will be used. Which one is undefined.
objc[46449]: Class CVWindow is implemented in both /Users/alice/opt/anaconda3/envs/new-env/lib/python3.10/site-packages/cv2/cv2.abi3.so (0x141ea2588) and /Users/alice/opt/anaconda3/envs/new-env/lib/python3.10/site-packages/mediapipe/.dylibs/libopencv_highgui.3.4.16.dylib (0x1161a8a68). One of the two will be used. Which one is undefined.
objc[46449]: Class CVView is implemented in both /Users/alice/opt/anaconda3/envs/new-env/lib/python3.10/site-packages/cv2/cv2.abi3.so (0x141ea25b0) and /Users/alice/opt/anaconda3/envs/new-env/lib/python3.10/site-packages/mediapipe/.dylibs/libopencv_highgui.3.4.16.dylib (0x1161a8a90). One of the two will be used.

In [2]:
class_name = ['cold', 'dizziness', 'allergic_reaction']

In [3]:
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils 

In [4]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) 
    image.flags.writeable = False                 
    results = model.process(image)                 
    image.flags.writeable = True                  
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) 
    return image, results

In [5]:
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks,
                              mp_holistic.FACEMESH_TESSELATION) 
    
    mp_drawing.draw_landmarks(image, results.pose_landmarks,
                              mp_holistic.POSE_CONNECTIONS) 
    
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks,
                              mp_holistic.HAND_CONNECTIONS) 
    
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks,
                              mp_holistic.HAND_CONNECTIONS) 

In [6]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    #print(pose)

    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    #print(face)

    left_hand = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    #print(left_hand)

    right_hand = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    #print(right_hand)
    
    return np.concatenate([pose, face, left_hand, right_hand])

In [7]:
def ex_key(name_file):
    list_data = []
    cap = cv2.VideoCapture(name_file)
    with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
        # Loop through video length aka sequence length
        for frame_num in range(90):

            # Read feed
            ret, frame = cap.read()

            # Make detections
            image, results = mediapipe_detection(frame, holistic)
            keypoints = extract_keypoints(results)
            list_data.append(keypoints)

            # Break gracefully
            if cv2.waitKey(10) & 0xFF == ord('q'):
                break

    cap.release()
    cv2.destroyAllWindows()
    arr_data = np.array(list_data)
    print(arr_data.shape)
    return arr_data

In [35]:
p_data = ex_key(os.path.join('videos','2222.mp4'))

(90, 1662)


In [36]:
scaler = MinMaxScaler(feature_range=(0,1))

In [37]:
p_data = scaler.fit_transform(p_data.reshape(p_data.shape[0], -1)).reshape(p_data.shape)

In [38]:
model = tf.keras.models.load_model('baseline_3class_7667.h5')

In [39]:
prediction = model.predict(tf.expand_dims(p_data, axis=0))

1/1 [==============================] - 0s 99ms/step


In [40]:
prediction

array([[0.9769927 , 0.01403888, 0.00896842]], dtype=float32)

In [41]:
np.argmax(prediction)

0

In [42]:
result = class_name[np.argmax(prediction)]

In [43]:
result

'cold'